In [ ]:
import os, sys, glob, shutil, json
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
import cv2

from PIL import Image
import numpy as np

from tqdm import tqdm, tqdm_notebook

import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

In [ ]:
class SVHNDataset(Dataset):
    def __init__(self, img_path, img_label, transform=None):
        self.img_path = img_path
        self.img_label = img_label 
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None

    def __getitem__(self, index):
        img = Image.open(self.img_path[index]).convert('RGB')

        if self.transform is not None:
            img = self.transform(img)
        
        # 设置最长的字符长度为5个
        lbl = np.array(self.img_label[index], dtype=np.int)
        lbl = list(lbl)  + (5 - len(lbl)) * [10]
        return img, torch.from_numpy(np.array(lbl[:5])).long()

    def __len__(self):
        return len(self.img_path)


In [ ]:
class RandomErasing(object):
    def __init__(self, p=0.5, sl=0.02, sh=0.4, r1=0.3, r2=3):
        self.p = p
        self.sl = sl
        self.sh = sh
        self.r1 = r1
        self.r2 = r2

    def __call__(self, img):
        
        if np.random.rand() > self.p:
            return img
        
        img = np.array(img)
        
        while True:
            img_h, img_w, img_c = img.shape

            img_area = img_h * img_w
            mask_area = np.random.uniform(self.sl, self.sh) * img_area
            mask_aspect_ratio = np.random.uniform(self.r1, self.r2)
            mask_w = int(np.sqrt(mask_area / mask_aspect_ratio))
            mask_h = int(np.sqrt(mask_area * mask_aspect_ratio))
            
            mask = np.random.rand(mask_h, mask_w, img_c) * 255

            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)
            right = left + mask_w
            bottom = top + mask_h
        
            if right <= img_w and bottom <= img_h:
                break
        
        img[top:bottom, left:right, :] = mask
        
        return Image.fromarray(img)



In [ ]:

train_path = glob.glob('../input/jiejingshibie/train/train/*.png')
train_path.sort()
train_json = json.load(open('../input/jiejingshibie/train.json'))



val_path = glob.glob('../input/jiejingshibie/val/val/*.png')
val_path.sort()
val_json = json.load(open('../input/jiejingshibie/val.json'))
path = train_path + val_path
train_path = path[8000:]
val_path = path[:8000]


train_label = [train_json[x]['label'] for x in train_json]
val_label = [val_json[x]['label'] for x in val_json]
label = train_label + val_label
train_label = label[8000:]
val_label = label[:8000]

print(len(train_path), len(train_label))
print(len(val_path), len(val_label))


train_loader = torch.utils.data.DataLoader(
    SVHNDataset(train_path, train_label,
                transforms.Compose([
                    transforms.Resize((108, 216)),
                    transforms.RandomCrop((100, 200)),
                    transforms.ColorJitter(0.3, 0.3, 0.2),
#                     RandomErasing(p=0.2, sh=0.2),
                    transforms.RandomGrayscale(p=0.2),
                    transforms.RandomRotation(5),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])), 
    batch_size=128, 
    shuffle=True, 
    num_workers=16,
)



val_loader = torch.utils.data.DataLoader(
    SVHNDataset(val_path, val_label,
                transforms.Compose([
                    transforms.Resize((100, 200)),
                    # transforms.RandomGrayscale(p=1),
                    # transforms.ColorJitter(0.3, 0.3, 0.2),
                    # transforms.RandomRotation(5),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])), 
    batch_size=128, 
    shuffle=False, 
    num_workers=16,
)

In [ ]:
class SVHN_Model1(nn.Module):
    def __init__(self):
        super(SVHN_Model1, self).__init__()
                
        model_conv = models.resnet50(pretrained=True)
        model_conv.avgpool = nn.AdaptiveAvgPool2d(1)
        model_conv = nn.Sequential(*list(model_conv.children())[:-1])
        self.cnn = model_conv
        self.fc1 = nn.Linear(2048, 11)
        self.fc2 = nn.Linear(2048, 11)
        self.fc3 = nn.Linear(2048, 11)
        self.fc4 = nn.Linear(2048, 11)
        self.fc5 = nn.Linear(2048, 11)
#         self.fc6 = nn.Linear(3072, 7)
#         self.hidden1 = nn.Sequential(
#             nn.Linear(2048, 3072),
#             nn.ReLU())
#         self.hidden2 = nn.Sequential(
#             nn.Linear(3072, 3072),
#             nn.ReLU())
        self.dr1 = nn.Dropout(0.5)
        self.dr2 = nn.Dropout(0.5)
        self.dr3 = nn.Dropout(0.5)
        self.dr4 = nn.Dropout(0.5)
        self.dr5 = nn.Dropout(0.5)
#         self.dr6 = nn.Dropout(0.5)
    def forward(self, img):        
        feat = self.cnn(img)
        # print(feat.shape)
        feat = feat.view(feat.shape[0], -1)
        c1 = self.dr1(self.fc1(feat))
        c2 = self.dr2(self.fc2(feat))
        c3 = self.dr3(self.fc3(feat))
        c4 = self.dr4(self.fc4(feat))
        c5 = self.dr5(self.fc5(feat))

        return c1, c2, c3, c4, c5

In [ ]:
def train(train_loader, model, criterion, optimizer):
    # 切换模型为训练模式
    model.train()
    train_loss = []
    
    for i, (input, target) in enumerate(train_loader):
        if use_cuda:
            input = input.cuda()
            target = target.cuda()
            
        c0, c1, c2, c3, c4 = model(input)
        loss = criterion(c0, target[:, 0])*1.2 + \
               criterion(c1, target[:, 1])*1.2 + \
               criterion(c2, target[:, 2])*1.1 + \
               criterion(c3, target[:, 3])*1.1 + \
               criterion(c4, target[:, 4])
        
        # loss /= 6
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i % 100 == 0:
            print(loss.item())
        
        train_loss.append(loss.item())
    return np.mean(train_loss)

def validate(val_loader, model, criterion):
    # 切换模型为预测模型
    model.eval()
    val_loss = []

    # 不记录模型梯度信息
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            if use_cuda:
                input = input.cuda()
                target = target.cuda()
            
            c0, c1, c2, c3, c4 = model(input)
            loss = criterion(c0, target[:, 0])*1.2 + \
                    criterion(c1, target[:, 1])*1.2 + \
                    criterion(c2, target[:, 2])*1.1 + \
                    criterion(c3, target[:, 3])*1.1 + \
                    criterion(c4, target[:, 4])
            # loss /= 6
            val_loss.append(loss.item())
    return np.mean(val_loss)

def predict(test_loader, model, tta=10):
    model.eval()
    test_pred_tta = None
    
    # TTA 次数
    for _ in range(tta):
        test_pred = []
    
        with torch.no_grad():
            for i, (input, target) in enumerate(test_loader):
                if use_cuda:
                    input = input.cuda()
                
                c0, c1, c2, c3, c4 = model(input)
                output = np.concatenate([
                    c0.cpu().data.numpy(), 
                    c1.cpu().data.numpy(),
                    c2.cpu().data.numpy(), 
                    c3.cpu().data.numpy(),
                    c4.cpu().data.numpy()], axis=1)
                test_pred.append(output)
        
        test_pred = np.vstack(test_pred)
        if test_pred_tta is None:
            test_pred_tta = test_pred
        else:
            test_pred_tta += test_pred
    
    return test_pred_tta

In [ ]:
model = SVHN_Model1()
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), 0.001)
best_loss = 1000.0

use_cuda = False
if use_cuda:
    model = model.cuda()

for epoch in range(30):
    if epoch <= 19:
        lr = 0.001
    else:
        lr = 0.0001
#     optimizer = torch.optim.RMSprop(model.parameters(), lr, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False)
    optimizer = torch.optim.Adam(model.parameters(), lr)
    train_loss = train(train_loader, model, criterion, optimizer)
    val_loss = validate(val_loader, model, criterion)
    
    val_label = [''.join(map(str, x)) for x in val_loader.dataset.img_label]
    val_predict_label = predict(val_loader, model, 1)
    val_predict_label = np.vstack([
        val_predict_label[:, :11].argmax(1),
        val_predict_label[:, 11:22].argmax(1),
        val_predict_label[:, 22:33].argmax(1),
        val_predict_label[:, 33:44].argmax(1),
        val_predict_label[:, 44:55].argmax(1),
    ]).T
    val_label_pred = []
    for x in val_predict_label:
        val_label_pred.append(''.join(map(str, x[x!=10])))
    
    val_char_acc = np.mean(np.array(val_label_pred) == np.array(val_label))
    
    print('Epoch: {0}, Train loss: {1} \t Val loss: {2}'.format(epoch, train_loss, val_loss))
    print(val_char_acc)
    # 记录下验证集精度
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), './model1.pt')

In [ ]:
model.load_state_dict(torch.load("model1.pt"))

test_path = glob.glob('../input/jiejingshibie/test_a/test_a/*.png')
test_path.sort()
test_label = [[1]] * len(test_path)
print(len(test_path), len(test_label))

test_loader = torch.utils.data.DataLoader(
    SVHNDataset(test_path, test_label,
                transforms.Compose([
                    transforms.Resize((100, 200)),
                    transforms.RandomGrayscale(p=1),
                    transforms.ColorJitter(0.3, 0.3, 0.2),
                    transforms.RandomRotation(5),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])), 
    batch_size=40, 
    shuffle=False, 
#     num_workers=10,
)

test_predict_label = predict(test_loader, model, 1)

test_label = [''.join(map(str, x)) for x in test_loader.dataset.img_label]
test_predict_label = np.vstack([
    test_predict_label[:, :11].argmax(1),
    test_predict_label[:, 11:22].argmax(1),
    test_predict_label[:, 22:33].argmax(1),
    test_predict_label[:, 33:44].argmax(1),
    test_predict_label[:, 44:55].argmax(1),
]).T

test_label_pred = []
for x in test_predict_label:
    test_label_pred.append(''.join(map(str, x[x!=10])))
    
import pandas as pd
df_submit = pd.read_csv('../input/jiejing-bianma-shibie/mchar_sample_submit_A.csv')
df_submit['file_code'] = test_label_pred
df_submit.to_csv('renset50tta.csv', index=False)

In [ ]:
model.load_state_dict(torch.load("model1.pt"))

test_path = glob.glob('../input/jiejingshibie/test_a/test_a/*.png')
test_path.sort()
test_label = [[1]] * len(test_path)
print(len(test_path), len(test_label))

test_loader = torch.utils.data.DataLoader(
    SVHNDataset(test_path, test_label,
                transforms.Compose([
                    transforms.Resize((100, 200)),
#                     transforms.RandomGrayscale(p=1),
#                     transforms.ColorJitter(0.3, 0.3, 0.2),
#                     transforms.RandomRotation(5),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])), 
    batch_size=40, 
    shuffle=False, 
#     num_workers=10,
)

test_predict_label = predict(test_loader, model, 1)

test_label = [''.join(map(str, x)) for x in test_loader.dataset.img_label]
test_predict_label = np.vstack([
    test_predict_label[:, :11].argmax(1),
    test_predict_label[:, 11:22].argmax(1),
    test_predict_label[:, 22:33].argmax(1),
    test_predict_label[:, 33:44].argmax(1),
    test_predict_label[:, 44:55].argmax(1),
]).T

test_label_pred = []
for x in test_predict_label:
    test_label_pred.append(''.join(map(str, x[x!=10])))
    
import pandas as pd
df_submit = pd.read_csv('../input/jiejing-bianma-shibie/mchar_sample_submit_A.csv')
df_submit['file_code'] = test_label_pred
df_submit.to_csv('renset50notta.csv', index=False)